# INFO 2950 Project Phase II

## Research Questions

## Data Collection and Cleaning Data

Raw Data was collected by downloading the delimited data from the study at this link, https://www.icpsr.umich.edu/web/ICPSR/studies/38050/summary and Git Large Language Storage was used to upload the raw files onto Github. 

After opening the original public_data tsv file, we observed that several variables were not available publicly and were only available in the restricted dataset which we did not have access to. These variables include NAME, URL_NAME, and PROJECT_PAGE_LOCATION_COUNTRY.  In addition, there were some rows that had missing values, particularly in the PROJECT_PAGE_LOCATION_STATE, and PROJECT_PAGE_LOCATION_COUNTY columns which may indicate that these are user-inputs that are optional. In addition, although documentation for each dataset was provided upon download, descriptions for qualitative variables like STATE were unclear in how they categorized each startup into the 4 possible states (successful, failed, suspended, canceled). 

After opening a portion of the original backer_location tsv file, we observed that there were many rows that had missing values in either/both the USER_LOCATION_STATE and the USER_LOCATION_COUNTRY column. In addition, Numbers, the application tried to open the raw data in, warned us that it can only show the first 999999 rows of data. We either have to separate the original tsv into multiple tsv files to look at all of the data or do a boolean statement or GROUPBY to extract the most relevant information (Note: the first 182,253 entries have 50 people backing the project or more). Due to these issues, and being unsure of what to GROUPBY, we set this dataset aside for now to analyze the data in the first dataset (public_data.tsv).

In the future, we plan to merge the two tsv files by PID (Project ID???) to gain more insights on how location influences the amount of funding raised for each geographic location.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import duckdb

We checked for duplicates in the public_data.tsv dataset by selecting the rows that appeared more than once according to the unique PID. Since there are no rows that appeared more than once, we know that there is no duplicate PIDs in the dataset.

In [8]:
public_data = pd.read_csv('data/public_data.tsv', delimiter='\t')
public_data.head()

duckdb.sql("SELECT PID, COUNT(PID) FROM public_data_tsv GROUP BY PID HAVING COUNT(PID) > 1").df()

/var/folders/px/nk9xdwpx6kn6_4khy3rv8xkm0000gn/T/ipykernel_83591/3563118418.py:1: DtypeWarning: Columns (9,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  public_data_tsv = pd.read_csv('data/public_data.tsv', delimiter='\t')


,PID,count(PID)


## Exploratory Data Analysis

## Data Description

## Data Limitations

## Questions For Reviewers